In [2]:
from __future__ import division, print_function, unicode_literals
import csv
import shutil
import os
import time
import requests
import subprocess
from subprocess import call
import numpy as np
import tensorflow as tf


In [3]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [4]:
def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

### RNN Experimnet 

In [6]:
import gym

In [7]:
import gym_foo

In [8]:
env = gym.make('foo-v7')

__init__


In [9]:
env.reset()

array([[38.19587973, 38.19587973,  1.        ,  0.5       , 19.09793986],
       [59.83143932, 59.83143932,  1.        ,  0.5       , 29.91571966],
       [38.19587973, 38.19587973,  1.        ,  0.5       , 19.09793986],
       [ 0.        ,  0.        ,  1.        ,  0.5       ,  0.        ]])

In [10]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.zeros(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

## Deep Q-Learning


In [11]:
env.action_space

Discrete(6)

In [12]:
possible_actions = [0, 1, 2, 3, 4, 5]    

In [13]:
 env.observation_space.shape[1]

5

In [14]:
tf.reset_default_graph()

# 1. Specify the network architecture
n_inputs = 20  # == 24
n_hidden = 10
n_outputs = len(possible_actions) # == 625
initializer = tf.variance_scaling_initializer()

# 2. Build the neural network
X = tf.placeholder(tf.float32, shape=[None, n_inputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.selu,
                         kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs,
                         kernel_initializer=initializer)
outputs = tf.nn.softmax(logits)

# 3. Select a random action based on the estimated probabilities
action_index = tf.squeeze(tf.multinomial(logits, num_samples=1), axis=-1)

# 4. Training
learning_rate = 0.01

y = tf.one_hot(action_index, depth=len(possible_actions))
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [15]:
def run_bipedal_walker(model_path=None, n_max_steps = 1000):
    env = gym.make("foo-v6")
    frames = []
    with tf.Session() as sess:
        if model_path is None:
            init.run()
        else:
            saver.restore(sess, model_path)
        obs = env.obs
        print(obs.shape)
        for step in range(n_max_steps):
            action_index_val = action_index.eval(feed_dict={X: obs.reshape(1, 20)})
            action = possible_actions[int(action_index_val)]
            print (action)
            obs, reward, done, info = env.step(action)
            if done:
                break
    env.close()
    return 0


In [91]:
run_bipedal_walker()

__init__
(4, 5)
2
[[9.70478904e+00 9.70478904e+00 1.00000000e+00 5.00000000e-01
  4.85239452e+00]
 [5.10447301e+01 5.10447301e+01 1.00000000e+00 5.00000000e-01
  2.55223650e+01]
 [9.70478904e+00 9.70478904e+00 1.00000000e+00 5.00000000e-01
  4.85239452e+00]
 [4.88659244e+03 4.88659244e+03 1.00000000e+00 5.00000000e-01
  2.44329622e+03]]
Mantain Cluster State S0
2
[[9.70478904e+00 9.70478904e+00 1.00000000e+00 5.00000000e-01
  4.85239452e+00]
 [5.10447301e+01 5.10447301e+01 1.00000000e+00 5.00000000e-01
  2.55223650e+01]
 [9.70478904e+00 9.70478904e+00 1.00000000e+00 5.00000000e-01
  4.85239452e+00]
 [4.88659244e+03 4.88659244e+03 1.00000000e+00 5.00000000e-01
  2.44329622e+03]]
Mantain Cluster State S0
2
[[9.70478904e+00 9.70478904e+00 1.00000000e+00 5.00000000e-01
  4.85239452e+00]
 [5.10447301e+01 5.10447301e+01 1.00000000e+00 5.00000000e-01
  2.55223650e+01]
 [9.70478904e+00 9.70478904e+00 1.00000000e+00 5.00000000e-01
  4.85239452e+00]
 [4.88659244e+03 4.88659244e+03 1.00000000e+00

0

In [ ]:
n_games_per_update = 10
n_max_steps = 100
n_iterations = 100
save_iterations = 10
discount_rate = 0.95
n_inputs=20
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        print("\rIteration: {}/{}".format(iteration + 1, n_iterations), end="")
        all_rewards = []
        all_gradients = []
        for game in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            obs = env.obs
            for step in range(n_max_steps):
                action_index_val, gradients_val = sess.run([action_index, gradients], feed_dict={X: obs.reshape(1, 20)})
                
                action = possible_actions[np.random.randint(0,5)]
                obs, reward, done, info = env.step(action)
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)

        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate=discount_rate)
        feed_dict = {}
        for var_index, gradient_placeholder in enumerate(gradient_placeholders):
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                      for game_index, rewards in enumerate(all_rewards)
                                          for step, reward in enumerate(rewards)], axis=0)
            feed_dict[gradient_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            saver.save(sess, "./microservice_policyGradiant.ckpt")

Iteration: 1/100[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.09793986]
 [93.32711153 93.32711153  1.          0.5        46.66355576]]
Add Node with attempt:  1
reward:  46.66355576294914 46.66355576294914 -45.66355576294914
[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.09793986]
 [93.32711153 93.32711153  1.          0.5        46.66355576]]
Delete Node with attempt:  2
reward:  46.66355576294914 46.66355576294914 -45.66355576294914
[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.09793986]
 [93.32711153 93.32711153  1.          0.5        46.66355576]]
Scale Service UP S2
<Response [200]>
46.66

0
reward:  46.66355576294914 46.66355576294914 -45.66355576294914
[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.09793986]
 [93.32711153 93.32711153  1.          0.5        46.66355576]]
Scale Service UP S2
<Response [200]>
46.66355576294914
[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.09793986]
 [93.32711153 93.32711153  1.          0.5        46.66355576]]
Add Node S4
/Users/baz/ieee-demo/addNode.sh
0
reward:  46.66355576294914 46.66355576294914 -45.66355576294914
[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.09793986]
 [93.32711153 93.32711153  1.          0.5        46.66355576]]
Scale Serv

0
reward:  46.66355576294914 46.66355576294914 -45.66355576294914
[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.09793986]
 [93.32711153 93.32711153  1.          0.5        46.66355576]]
Delete Node S4
/Users/baz/ieee-demo/deleteNode.sh
0
reward:  46.66355576294914 46.66355576294914 -45.66355576294914
Iteration: 3/100[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.09793986]
 [93.32711153 93.32711153  1.          0.5        46.66355576]]
Add Node S4
/Users/baz/ieee-demo/addNode.sh
0
reward:  46.66355576294914 46.66355576294914 -45.66355576294914
[[38.19587973 38.19587973  1.          0.5        19.09793986]
 [59.83143932 59.83143932  1.          0.5        29.91571966]
 [38.19587973 38.19587973  1.          0.5        19.0979398

In [100]:
history = run_bipedal_walker("./microservice_policyGradiant.ckpt")



__init__


ValueError: The passed save_path is not a valid checkpoint: ./microservice_policyGradiant.ckpt